# Sportradar: Schedule and team fetching

In [1]:
print("hello world")

hello world


In [2]:
import sys
import os
from django.conf import settings

In [5]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)

In [6]:
# Set up Django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "hooperstats.settings")
import django
django.setup()

In [7]:
# Now we can import your utility files
from api.sportradar_utils import get_sportradar_data
from api.supabase_utils import get_supabase_client

In [8]:
# Get Supabase client
supabase = get_supabase_client()

In [9]:
# Delete dummy data
dummy_teams = ['Los Angeles Lakers', 'Boston Celtics', 'Golden State Warriors', 'Chicago Bulls']
for team in dummy_teams:
    result = supabase.table('teams').delete().eq('name', team).execute()
    print(f"Deleted {result.data} rows for {team}")

Deleted [{'id': '653215a5-b6ba-4c34-93ea-be1a6e368ec8', 'name': 'Los Angeles Lakers'}] rows for Los Angeles Lakers
Deleted [{'id': '17353121-14ec-446d-acad-65d180731cfc', 'name': 'Boston Celtics'}] rows for Boston Celtics
Deleted [{'id': 'd1f7a0d9-5bcf-436d-861d-d364ad8bccde', 'name': 'Golden State Warriors'}] rows for Golden State Warriors
Deleted [{'id': '4374eda1-1ff8-4f0c-8415-55e4c69aa6ff', 'name': 'Chicago Bulls'}] rows for Chicago Bulls


In [10]:
# Verify deletion
remaining_teams = supabase.table('teams').select('*').execute()
print("Remaining teams:")
print(remaining_teams.data)

Remaining teams:
[]


In [13]:
# Fetch schedule data
schedule_data = get_sportradar_data('games/2023/REG/schedule.json')

In [17]:
print("Schedule data:")


Schedule data:


In [15]:
import pandas as pd

In [ ]:
# Extract teams
teams = {}
for game in schedule_data['games']:
    home_team = game['home']
    away_team = game['away']
    teams[home_team['id']] = home_team['name']
    teams[away_team['id']] = away_team['name']

In [18]:
# Extract teams
teams = {}
# When you assign a value to a dictionary key, it either creates a new entry (if the key didn't exist) or overwrites the existing value (if the key already existed).
# Since team IDs are unique, even if a team appears in multiple games, it will only be added to the dictionary once.

for game in schedule_data['games']:
    home_team = game['home']
    away_team = game['away']
    teams[home_team['id']] = home_team['name']
    teams[away_team['id']] = away_team['name']

# Convert to DataFrame for easy viewing
teams_df = pd.DataFrame(list(teams.items()), columns=['id', 'name'])
print(f"Total teams found: {len(teams_df)}")
teams_df

Total teams found: 36


,id,name
0,583ed102-fb46-11e1-82cb-f4ce4684ea4c,Denver Nuggets
1,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers
2,583ec825-fb46-11e1-82cb-f4ce4684ea4c,Golden State Warriors
3,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns
4,583ec97e-fb46-11e1-82cb-f4ce4684ea4c,Charlotte Hornets
5,583ecb8f-fb46-11e1-82cb-f4ce4684ea4c,Atlanta Hawks
6,583ec7cd-fb46-11e1-82cb-f4ce4684ea4c,Indiana Pacers
7,583ec8d4-fb46-11e1-82cb-f4ce4684ea4c,Washington Wizards
8,583ec70e-fb46-11e1-82cb-f4ce4684ea4c,New York Knicks
9,583eccfa-fb46-11e1-82cb-f4ce4684ea4c,Boston Celtics


In [19]:
# List of team IDs to remove
teams_to_remove = [
    '2674a061-2cb1-4a0b-b0b6-e237ff267f45',
    '739e187c-5d96-477d-a2fb-5f40439138fc',
    '849c6b54-2cdc-4242-b194-7f7edd2ee341',
    'd64e9292-0b2b-4c21-a623-fcadad5496f5',
    '592d3144-895b-43e9-ab15-e84666a845d5',
    '787ff06f-f0cc-484e-8114-04e0cdc444fa'
]

# Remove the teams
teams_df = teams_df[~teams_df['id'].isin(teams_to_remove)]

# Reset the index if needed
teams_df = teams_df.reset_index(drop=True)

# Print the updated DataFrame
print(f"Total teams after removal: {len(teams_df)}")
print(teams_df)

Total teams after removal: 30
                                      id                    name
0   583ed102-fb46-11e1-82cb-f4ce4684ea4c          Denver Nuggets
1   583ecae2-fb46-11e1-82cb-f4ce4684ea4c      Los Angeles Lakers
2   583ec825-fb46-11e1-82cb-f4ce4684ea4c   Golden State Warriors
3   583ecfa8-fb46-11e1-82cb-f4ce4684ea4c            Phoenix Suns
4   583ec97e-fb46-11e1-82cb-f4ce4684ea4c       Charlotte Hornets
5   583ecb8f-fb46-11e1-82cb-f4ce4684ea4c           Atlanta Hawks
6   583ec7cd-fb46-11e1-82cb-f4ce4684ea4c          Indiana Pacers
7   583ec8d4-fb46-11e1-82cb-f4ce4684ea4c      Washington Wizards
8   583ec70e-fb46-11e1-82cb-f4ce4684ea4c         New York Knicks
9   583eccfa-fb46-11e1-82cb-f4ce4684ea4c          Boston Celtics
10  583ed157-fb46-11e1-82cb-f4ce4684ea4c           Orlando Magic
11  583ecb3a-fb46-11e1-82cb-f4ce4684ea4c         Houston Rockets
12  583ecda6-fb46-11e1-82cb-f4ce4684ea4c         Toronto Raptors
13  583eca2f-fb46-11e1-82cb-f4ce4684ea4c  Minnesota Timberwo

## Populating teams table in supabase

In [20]:
# Convert DataFrame to a list of dictionaries
teams_data = teams_df.to_dict('records')

In [22]:
# Clear existing data (optional, use with caution)

# Check if there are any records in the teams table
existing_teams = supabase.table('teams').select('id').execute()

if existing_teams.data:
    # If there are records, delete them all
    delete_result = supabase.table('teams').delete().execute()
    print(f"Deleted {len(delete_result.data)} existing team records.")
else:
    # If there are no records, inform that the table is empty
    print("The teams table is already empty.")

The teams table is already empty.


In [23]:
# Verify the table state after operation
verification = supabase.table('teams').select('id').execute()
print(f"Current number of records in teams table: {len(verification.data)}")

Current number of records in teams table: 0


In [24]:
# Insert data into Supabase
result = supabase.table('teams').insert(teams_data).execute()

# Check the result
if result.data:
    print(f"Successfully inserted {len(result.data)} teams.")
else:
    print("Failed to insert teams.")
    print("Error:", result.error)

Successfully inserted 30 teams.


In [25]:
# Verify the insertion
fetched_teams = supabase.table('teams').select('*').execute()
fetched_df = pd.DataFrame(fetched_teams.data)

print(f"Total teams in Supabase: {len(fetched_df)}")
print(fetched_df)


Total teams in Supabase: 30
                                      id                    name
0   583ed102-fb46-11e1-82cb-f4ce4684ea4c          Denver Nuggets
1   583ecae2-fb46-11e1-82cb-f4ce4684ea4c      Los Angeles Lakers
2   583ec825-fb46-11e1-82cb-f4ce4684ea4c   Golden State Warriors
3   583ecfa8-fb46-11e1-82cb-f4ce4684ea4c            Phoenix Suns
4   583ec97e-fb46-11e1-82cb-f4ce4684ea4c       Charlotte Hornets
5   583ecb8f-fb46-11e1-82cb-f4ce4684ea4c           Atlanta Hawks
6   583ec7cd-fb46-11e1-82cb-f4ce4684ea4c          Indiana Pacers
7   583ec8d4-fb46-11e1-82cb-f4ce4684ea4c      Washington Wizards
8   583ec70e-fb46-11e1-82cb-f4ce4684ea4c         New York Knicks
9   583eccfa-fb46-11e1-82cb-f4ce4684ea4c          Boston Celtics
10  583ed157-fb46-11e1-82cb-f4ce4684ea4c           Orlando Magic
11  583ecb3a-fb46-11e1-82cb-f4ce4684ea4c         Houston Rockets
12  583ecda6-fb46-11e1-82cb-f4ce4684ea4c         Toronto Raptors
13  583eca2f-fb46-11e1-82cb-f4ce4684ea4c  Minnesota Timberwolv